In [2]:
import sklearn
import csv
import numpy as np
import re
import pandas as pd
import unicodedata
from nltk.tokenize import word_tokenize
from sklearn.decomposition import TruncatedSVD
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import Normalizer
from nltk.corpus import stopwords
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
from nltk.stem.snowball import FrenchStemmer
from nltk.tokenize import word_tokenize

#https://www.datacamp.com/community/tutorials/discovering-hidden-topics-python


# Pre-processing

In [3]:
df = pd.read_excel(r"C:\Users\x194356\Desktop\Intelligent_Data_Process\BT\Enrichir\input_file.xlsx")
#df=df.rename(columns = {'is linked to [Local Data Element] > Name':'Donnee_metier'})
df.head()

,Description,Nom_attribut,is linked to [Local Data Element] > Name
0,NaN,Date d'achat du vehicule,2ème meilleure note de l'émetteur en date d'in...
1,Activités : Banque Agence,Etat du devis,2ème meilleure note de l'émission en date d'in...
2,Activités : Code courrier contact,Montant total de la surprime,2ème meilleure note du réassureur à la date d'...
3,Activités : Document contact,Déclaration de sinistre dans les 36 dernier mois,Actif ou passif sous-jacent du dérivé (si micr...
4,Activités : Date de création contact,statut du sous contrat,Activité du risque à assurer


In [4]:
col1=df.columns[0]
col2=df.columns[1]

proxy1=["N° Personne Sogessur","Numéro de personne","N° de personne,SI - N° Personne","SI - N° Personne Sogessur","N° Personne"]
for i in proxy1:
    df[col1] =df[col1].str.replace(i,"Numéro de client assuré")
    
proxy2=["CO - Coefficient Commercial","CO - Coefficient Commercial (Réduction Salariés)"]
for i in proxy2:
    df[col1] =df[col1].str.replace(i,"Ancienne réduction salariés (RED)")
    
proxy3=["SI - Date Saisie Montant","SI - Montant Provisions Restantes","Numéro de Police SOGESSUR","N° Devis","ALD - N° Police SOG","SI - Amount Incurred","CO - Acceptation FID O/N","SI - Montant Total Paiements","CO - CSP de l'assuré principal","SI - City of Accident"]
ch=["Date mouvement technique sinistre","Montant de la provision cédée restante","Numéro de contrat","Numéro de contrat","Numéro de contrat","Provision brute à la garantie du sinistre","Coefficient de fidélisation auto","Règlement net de recours à la garantie du sinistre","Catégorie socio-professionnelle de l'assuré","Ville du sinistre MRH"]
for i,j in zip(proxy3,ch):
    df['Description'] =df['Description'].str.replace(i,j)

proxy4=["CO - Commercial Premium Amount","CO - Gross Premium","CO - Montat Prime HT"]
for i in proxy4:
    df['Description'] =df['Description'].str.replace(i,"Montant de la prime commerciale d'assurance hors taxes en devise de reporting")
print(len(df[col1]))
df[col2] = df[col2][(~df[col2].duplicated()) | df[col2].isna()] #drop dup de col2 ignorant les Nan value 
df=df.apply(lambda x: x.astype(str).str.lower())       
df[col1] =df[col1].str.replace('n°client','numero client')
df[col1] =df[col1].str.replace('n°','numero')
df[col1]=df[col1].str.replace("d'", '')
#df[col1]=df[col1].str.replace('\d+', '')
df[col1] =df[col1].str.replace('[^\w\s]',' ')

df[col2]=df[col2].str.replace("d'", '')
#df[col2]=df[col2].str.replace('\d+', '')
df[col2] =df[col2].str.replace('[^\w\s]',' ')    

l1=['number','region','contract','name','customer','guarantee','address',"claim","closed","opened","status","HT","amendment","somme","crmr","csdp","sdp","product"]
l2=['numero','pays','contrat','nom','client','garantie','adresse',"sinistre","cloture","ouverture","etat","Hors Taxes","avenant","montant","crm recalculé","coefficient de surveillance du portefeuille","surveillance du portefeuille","produit"]
for i,j in zip(l1,l2):
    df[col1] =df[col1].str.replace(i,j)
    
#df to list
example_des=df.reset_index()[col1].values.tolist()
example_m=df.reset_index()[col2].values.tolist()  
example_m = [i for i in example_m if str(i) != 'nan']
example_des = [i for i in example_des if str(i) != 'nan']
example_des=[i.lstrip() for i in example_des]
#example_m = list(dict.fromkeys(example_m)) 


9355


In [5]:
len(example_m)

104

In [6]:
len(example_des)

9287

In [41]:
example_m

['date de naissance du conjoint',
 'description conducteur',
 'prénom conjoint',
 'code marche',
 'code motif devis',
 'code offre tarif',
 'code particularité contrat',
 'code motif résiliation compagnie précédente',
 'description forcage hors norme',
 'contrat nanti',
 'flag contrat progogé',
 'cotisation mensuelle ttc',
 'date effet',
 'date de clôture',
 'type vente',
 'date horizon',
 'code forcage ',
 'lien de parenté',
 'code banque gestionnaire',
 'total prélevé origine',
 'souscripteur',
 'coordonnées banque prélèvement',
 'déclaration du numéro de contrat parent',
 'antériorité conducteur',
 'cause du sinistre',
 'localité dans l adresse',
 'numéro voie bénéficiaire',
 'description du véhicule',
 'numéro de la voie dans l adresse',
 'libellé type contrat',
 'complément identification du point géographique',
 'montant total prime',
 'statut courant',
 'code distributeur de l adhérent',
 'numéro adhérent',
 'code insee',
 'code de distribution courrier',
 'type adresse de l hab

In [42]:
example_des

['activités   banque agence',
 'activités   code courrier contact',
 'activités   document contact',
 'activités   date de création contact',
 'activités   date d échéance contact',
 'activités   heure de création contact',
 'activités   heure d échéance contact',
 'activités   heure de traitement',
 'activités   date de traitement',
 'activités   id interne client',
 'activités   interlocuteur contact',
 'activités   trigramme créateur contact',
 'activités   traité par  trigramme ',
 'activités   motif contact',
 'activités   motif de réclamation',
 'activités   nature contact',
 'activités   numero de contact',
 'activités   origine contact',
 'activités   priorité contact',
 'activités   souscription portefuille',
 'activités   statut contact',
 'activités   accord encadrement',
 'activités   accord re',
 'activités   agence sg',
 'activités   demandeur dérogation ou forcage',
 'activités   description',
 'activités   date de création',
 'activités   date début dérogation',
 'activ

# CHUNKS

In [1]:
def untokenize(words):
    text = ' '.join(words) 
    return text.strip()

def accent_remove(s):

    text = ''.join((c for c in unicodedata.normalize('NFD', s) if unicodedata.category(c) != 'Mn')) 
    return text

path=r'C:\Users\x194356\Desktop\Intelligent_Data_Process\BT\Enrichir'

with open(path+"\outputt_file1.csv", mode='w',newline='') as file:
    writer = csv.writer(file, delimiter=';', quotechar='"', quoting=csv.QUOTE_MINIMAL)
    writer.writerow(["Data1","Data2","Correlation score"])
    
chunk_size=3000
chunks = [example_des[x:x+chunk_size] for x in range(0, len(example_des),chunk_size)]

for i,j in zip(chunks,range(len(chunks))):
    example=example_m+i
    
    print(len(example))
    
    df = pd.DataFrame({'example':example})
    df['example']= [word_tokenize(entry) for entry in df['example']]
    stopWords = set(stopwords.words('French'))
    stopWords_ang = set(stopwords.words('English'))
    l=["-","d","co","si"]
    df['example']=df['example'].apply(lambda x: [item for item in x if item not in stopWords])
    df['example']=df['example'].apply(lambda x: [item for item in x if item not in stopWords_ang])
    df['example']=df['example'].apply(lambda x: [item for item in x if item not in l])
    #stemming_frensh


    stemmer = FrenchStemmer()
    stem_list=[]
    for i in df['example']:
        stem_list.append ([stemmer.stem(word) for word in i])

    df['example']=[untokenize(entry) for entry in stem_list]
    df['example']=[accent_remove(entry) for entry in df['example']]
    df['example'] =df['example'].str.replace(' +', ' ')
    example=df.values.tolist()
    flat_list = lambda l: [item for sublist in l for item in sublist]
    example=[i.lstrip() for i in df['example']]
    vectorizer = TfidfVectorizer(use_idf=True,smooth_idf=True,sublinear_tf=True)
    dtm = vectorizer.fit_transform(example)
    from scipy.sparse import csr_matrix
    X_sparse = csr_matrix(dtm)
    dtm = dtm.astype(float)
    lsa = TruncatedSVD(min(X_sparse.shape)-1, algorithm = 'arpack')
    dtm_lsa = lsa.fit_transform(dtm)
    dtm_lsa = Normalizer(copy=False).fit_transform(dtm_lsa)

    df = pd.read_excel(path+"\input_file.xlsx")
    #df=df.rename(columns = {'is linked to [Local Data Element] > Name':'Donnee_metier'})
    #example_des_b=df.reset_index()['Description'].values.tolist()
    
    #Label des Données brutes
    df[col2] = df[col2][(~df[col2].duplicated()) | df[col2].isna()] 
    #df to list
    example_des_b=df.reset_index()[col1].values.tolist()
    example_des_b = [i for i in example_des_b if str(i) != 'nan']
    example_m_b=df.reset_index()[col2].values.tolist()
    example_m_b = list(dict.fromkeys(example_m_b))

    
    example_m_b = [i for i in example_m_b if str(i) != 'nan']
    chunks_b= [example_des_b[x:x+chunk_size] for x in range(0, len(example_des_b), chunk_size)]
    example_num_b=example_m_b+chunks_b[j]

    
    similarity = np.asarray(np.asmatrix(dtm_lsa) * np.asmatrix(dtm_lsa).T)
    CrossValid=pd.DataFrame(similarity,index=example_num_b,columns=example_num_b)
    Matrice_M_Phy=CrossValid.iloc[len(example_m_b):, 0:len(example_m_b)]
    with open(path+"\outputt_file1.csv", mode='a',newline='') as file:       
        writer = csv.writer(file, delimiter=';', quotechar='"', quoting=csv.QUOTE_MINIMAL)
        for index, row in Matrice_M_Phy.iterrows():  
            writer.writerow([index,row.idxmax(),round(row.loc[row.idxmax()],4)])


NameError: name 'csv' is not defined

In [ ]:
"\output{}.csv".format(j)